# Collaborative Filtering Recommendations using Original Values

In [35]:
import pandas as pd
import numpy as np
import implicit

from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix

In [36]:
train = pd.read_csv("data\\train-plays.csv")
test = pd.read_csv("data\\test-plays.csv")
game_coding = pd.read_csv("data\\game-coding.csv")

In [37]:
train.shape[0]

36447

## Format training data

In [38]:
game_user = train.pivot(index="game_id", columns="user_id", values="amount")

In [39]:
game_user = game_user.fillna(0)

In [40]:
game_user_sparse = csr_matrix(game_user)

In [41]:
user_game = game_user.T

In [42]:
user_game_sparse = csr_matrix(user_game)

In [43]:
user_ids = train['user_id'].unique()

## Alternating Least Squares

In [44]:
als = implicit.als.AlternatingLeastSquares(128)

In [45]:
als.fit(game_user_sparse)

100%|██████████| 15/15 [00:00<00:00, 37.77it/s]


In [46]:
# rows_list = list()
# games_recommended = set()
# for user_id in user_ids:
#     results_dict = dict()
#     results_dict['user_id'] = user_id
#     test_df = test[test['user_id'] == user_id]
#     recommendations_df = pd.DataFrame(als.recommend(user_id, user_game_sparse))
#     recommended = test_df.merge(recommendations_df, left_on="game_id", right_on=0)
#     results_dict['num_recommended'] = recommended.shape[0]
#     results_dict['total_to_recommend'] = test_df.shape[0]
#     rows_list.append(results_dict)
#     games_recommended.update(recommended["game_id"].unique())

In [47]:
game_recs = np.ndarray.flatten(als.recommend_all(user_game_sparse))
user_10 = np.repeat(user_game.index, 10)
recommendations = pd.DataFrame({"user_id":user_10, "game_id":game_recs})

100%|██████████| 1160/1160 [00:00<00:00, 38660.25it/s]


In [48]:
results = recommendations.merge(test, on="user_id")

In [49]:
results['recommended'] = np.where(results.game_id_x == results.game_id_y, 1, 0)

In [50]:
results[results.recommended == 1].shape[0] / test.shape[0]

0.19655172413793104

In [53]:
np.unique(game_recs).size / train.game_id.unique().size

0.7046979865771812